# One class SVM and Isolation Forest for detecting fraudulent transaction.

using One class SVM and Isolation forest for clasification problem.
The task is to predict when a bank transaction is fraudulent or not. 
It must be noted that the Fraudulent class is highly imbalanced with respect to the whole data set.
It represents less that 0.2% of the total number of samples, therefore the classification task is very challenging
and the common performance metrics for classification problems are meaningless for highly imbalanced datasets.

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm

# Preprocess data

In [48]:
df=pd.read_csv('Data/classnofraud.csv',header=None)

In [49]:
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284310,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284311,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284312,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284313,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [50]:
X=df.values

In [51]:
X

array([[ 0.00000000e+00, -1.35980713e+00, -7.27811730e-02, ...,
        -2.10530530e-02,  1.49620000e+02,  0.00000000e+00],
       [ 0.00000000e+00,  1.19185711e+00,  2.66150712e-01, ...,
         1.47241690e-02,  2.69000000e+00,  0.00000000e+00],
       [ 1.00000000e+00, -1.35835406e+00, -1.34016307e+00, ...,
        -5.97518410e-02,  3.78660000e+02,  0.00000000e+00],
       ...,
       [ 1.72788000e+05,  1.91956501e+00, -3.01253846e-01, ...,
        -2.65608290e-02,  6.78800000e+01,  0.00000000e+00],
       [ 1.72788000e+05, -2.40440050e-01,  5.30482513e-01, ...,
         1.04532821e-01,  1.00000000e+01,  0.00000000e+00],
       [ 1.72792000e+05, -5.33412522e-01, -1.89733337e-01, ...,
         1.36489140e-02,  2.17000000e+02,  0.00000000e+00]])

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_train,X_test_val=train_test_split(X,test_size=0.3,random_state=0)

In [54]:
X_train

array([[ 1.35902000e+05, -5.79816915e-01,  4.79066172e-01, ...,
         1.91097095e-01,  4.99000000e+00,  0.00000000e+00],
       [ 1.72731000e+05,  2.01398681e+00,  5.73802092e-01, ...,
        -2.54098650e-02,  4.99000000e+01,  0.00000000e+00],
       [ 1.64885000e+05,  1.91202426e+00,  2.70946750e-02, ...,
        -6.63312510e-02,  5.88200000e+01,  0.00000000e+00],
       ...,
       [ 7.67030000e+04,  1.21206092e+00, -3.66070161e-01, ...,
         1.61300400e-03,  1.00000000e+00,  0.00000000e+00],
       [ 9.74290000e+04,  1.93602827e+00, -3.91544089e-01, ...,
        -3.82188150e-02,  1.49500000e+01,  0.00000000e+00],
       [ 7.49870000e+04,  1.52743267e+00, -4.61259769e-01, ...,
        -2.16501270e-02,  1.00000000e+00,  0.00000000e+00]])

In [55]:
X_test,X_val=train_test_split(X_test_val,test_size=0.5,random_state=0)

In [56]:
X_fraud_val_test=pd.read_csv('Data/classfraud.csv',header=None).values

In [57]:
X_fraud_val,X_fraud_test=train_test_split(X_fraud_val_test,test_size=0.5,random_state=0)

In [58]:
X_test=np.concatenate((X_test,X_fraud_test))

In [59]:
np.shape(X_test)

(42893, 31)

In [60]:
X_val=np.concatenate((X_val,X_fraud_val))

In [61]:
y_train=X_train[:,30]
y_test=X_test[:,30]
y_val=X_val[:,30]

In [62]:
print(y_train)

[0. 0. 0. ... 0. 0. 0.]


In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [64]:
sc=StandardScaler() #instantiate StandardScaler
minimum=np.min(X_train[:,0])
maximum=np.max(X_train[:,0])
X_train[:,0]=(X_train[:,0]-minimum)/(maximum-minimum)
X_test[:,0]=(X_test[:,0]-minimum)/(maximum-minimum)
X_val[:,0]=(X_val[:,0]-minimum)/(maximum-minimum)

In [65]:
X_train

array([[ 7.86506320e-01, -5.79816915e-01,  4.79066172e-01, ...,
         1.91097095e-01,  4.99000000e+00,  0.00000000e+00],
       [ 9.99646974e-01,  2.01398681e+00,  5.73802092e-01, ...,
        -2.54098650e-02,  4.99000000e+01,  0.00000000e+00],
       [ 9.54239780e-01,  1.91202426e+00,  2.70946750e-02, ...,
        -6.63312510e-02,  5.88200000e+01,  0.00000000e+00],
       ...,
       [ 4.43903653e-01,  1.21206092e+00, -3.66070161e-01, ...,
         1.61300400e-03,  1.00000000e+00,  0.00000000e+00],
       [ 5.63851336e-01,  1.93602827e+00, -3.91544089e-01, ...,
        -3.82188150e-02,  1.49500000e+01,  0.00000000e+00],
       [ 4.33972638e-01,  1.52743267e+00, -4.61259769e-01, ...,
        -2.16501270e-02,  1.00000000e+00,  0.00000000e+00]])

In [66]:
X_train[:,1:30]=sc.fit_transform(X_train[:,1:30])  # first fit StandardScaler to the training inputs
X_test[:,1:30]=sc.transform(X_test[:,1:30])
X_val[:,1:30]=sc.transform(X_val[:,1:30])

In [67]:
np.shape(X_train)

(199020, 31)

In [68]:
y_train[y_train==1]=-1
y_train[y_train==0]=1
y_test[y_test==1]=-1
y_test[y_test==0]=1
y_val[y_val==1]=-1
y_val[y_val==0]=1

# One class SVM

In [69]:
import time
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.04)
std=time.time()
clf.fit(X_train[:,1:30])
print("Time for One class SVM predicting: %.3f" % (time.time() - std)) 

Time for One class SVM predicting: 156.861


In [70]:
std=time.time()
y_pred_train = clf.predict(X_train[:,1:30])
y_pred_test = clf.predict(X_test[:,1:30])
y_pred_val = clf.predict(X_val[:,1:30])
n_error_train = y_pred_train[y_pred_train !=y_train].size
n_error_test = y_pred_test[y_pred_test != y_test].size
n_error_val = y_pred_val[y_pred_val !=y_val].size
print("Time for One class SVM predicting: %.3f" % (time.time() - std)) 

Time for One class SVM predicting: 121.411


In [71]:
print("error train: %3f ; errors test : %3f ; "
    "errors val: %3f"
    % (n_error_train/np.shape(X_train)[0], n_error_test/np.shape(X_test)[0], n_error_val/np.shape(X_val)[0]))

error train: 0.007240 ; errors test : 0.014944 ; errors val: 0.012985


In [72]:
y_train[y_train==1]=0
y_train[y_train==-1]=1
y_test[y_test==1]=0
y_test[y_test==-1]=1
y_val[y_val==1]=0
y_val[y_val==-1]=1

In [73]:
y_pred_train[y_pred_train==1]=0
y_pred_train[y_pred_train==-1]=1
y_pred_test[y_pred_test==1]=0
y_pred_test[y_pred_test==-1]=1
y_pred_val[y_pred_val==1]=0
y_pred_val[y_pred_val==-1]=1

In [74]:
from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_val, y_pred_val))
print(roc_auc_score(y_test, y_pred_test))

0.9166795343389563
0.899528193349793


In [75]:
from sklearn.metrics import precision_recall_curve
precision_val, recall_val, thresholds_val = precision_recall_curve(y_val, y_pred_val)
precision_test, recall_test, thresholds_test = precision_recall_curve(y_test, y_pred_test)

In [76]:
print(precision_val)
print(precision_test)

[0.00573507 0.28610729 1.        ]
[0.0057352  0.25157233 1.        ]


In [77]:
from sklearn.metrics import auc
auc_score_val = auc(recall_val, precision_val)
auc_score_test = auc(recall_test, precision_test)

In [78]:
print(auc_score_val,auc_score_test)

0.5662608251534696 0.532826446593577


In [79]:
from sklearn.metrics import average_precision_score
average_precision_1_test = average_precision_score(y_test, y_pred_test)
average_precision_1_val = average_precision_score(y_val, y_pred_val)
print('Average precision-recall score validation set: {0:0.5f}'.format(
      average_precision_1_val))
print('Average precision-recall score test set : {0:0.5f}'.format(
      average_precision_1_test))

Average precision-recall score validation set: 0.24280
Average precision-recall score test set : 0.20560


In [80]:
from sklearn.metrics import f1_score
score_test= f1_score(y_test, y_pred_test)
score_val= f1_score(y_val, y_pred_val)
print('F1 Score test: %.5f' % score_test)
print('F1 Score validation test: %.5f' % score_val)

F1 Score test: 0.38425
F1 Score validation test: 0.42754


In [83]:
from sklearn.metrics import classification_report

print("Classification report for OCSVM classifier %s:\n%s\n"
      % (clf, classification_report(y_test, y_pred_test)))

Classification report for OCSVM classifier OneClassSVM(gamma=0.04, nu=0.001):
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99     42647
         1.0       0.25      0.81      0.38       246

    accuracy                           0.99     42893
   macro avg       0.63      0.90      0.69     42893
weighted avg       0.99      0.99      0.99     42893




# Isolation forest

In [87]:
from sklearn.ensemble import IsolationForest
nus=np.arange(0.001,0.011,0.001)
n_ests=[100,200,500]
print(nus)
print(n_ests)
for nu in nus:
    for n_est in n_ests:
        clf2 =IsolationForest(contamination=nu,n_estimators=n_est,random_state=42) #fit isolation forest to data set
        stime=time.time()
        clf2.fit(X_train[:,1:30])
        print("contamination="+str(nu)+","+"n_estimators="+str(n_est))
        print("Time for Isolation Forest fitting: %.3f" % (time.time() - stime))
        std=time.time()
        y_pred_train_2= clf2.predict(X_train[:,1:30])
        y_pred_test_2 = clf2.predict(X_test[:,1:30])
        y_pred_val_2= clf2.predict(X_val[:,1:30])
        y_pred_train_2[y_pred_train_2==1]=0
        y_pred_train_2[y_pred_train_2==-1]=1
        y_pred_test_2[y_pred_test_2==1]=0
        y_pred_test_2[y_pred_test_2==-1]=1
        y_pred_val_2[y_pred_val_2==1]=0
        y_pred_val_2[y_pred_val_2==-1]=1
        average_precision_2_test= average_precision_score(y_test, y_pred_test_2)
        average_precision_2_val= average_precision_score(y_val, y_pred_val_2)
        print('Average precision-recall score test: {0:0.5f}'.format(average_precision_2_test))
        print('Average precision-recall score validation: {0:0.5f}'.format(average_precision_2_val))

[0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01 ]
[100, 200, 500]
contamination=0.001,n_estimators=100
Time for Isolation Forest fitting: 20.984
Average precision-recall score test: 0.07837
Average precision-recall score validation: 0.08409
contamination=0.001,n_estimators=200
Time for Isolation Forest fitting: 42.188
Average precision-recall score test: 0.08638
Average precision-recall score validation: 0.09128
contamination=0.001,n_estimators=500
Time for Isolation Forest fitting: 104.532
Average precision-recall score test: 0.08752
Average precision-recall score validation: 0.08618
contamination=0.002,n_estimators=100
Time for Isolation Forest fitting: 20.689
Average precision-recall score test: 0.13840
Average precision-recall score validation: 0.11752
contamination=0.002,n_estimators=200
Time for Isolation Forest fitting: 41.463
Average precision-recall score test: 0.13736
Average precision-recall score validation: 0.13068
contamination=0.002,n_estimators=500
Time for 

In [90]:
nus=[0.01]
n_ests=[500,1000,2000,3000,4000,5000,10000]
print(nus)
print(n_ests)
for nu in nus:
    for n_est in n_ests:
        clf2 =IsolationForest(contamination=nu,n_estimators=n_est,random_state=42) #fit isolation forest to data set
        stime=time.time()
        clf2.fit(X_train[:,1:30])
        print("contamination="+str(nu)+","+"n_estimators="+str(n_est))
        print("Time for Isolation Forest fitting: %.3f" % (time.time() - stime))
        std=time.time()
        y_pred_train_2= clf2.predict(X_train[:,1:30])
        y_pred_test_2 = clf2.predict(X_test[:,1:30])
        y_pred_val_2= clf2.predict(X_val[:,1:30])
        y_pred_train_2[y_pred_train_2==1]=0
        y_pred_train_2[y_pred_train_2==-1]=1
        y_pred_test_2[y_pred_test_2==1]=0
        y_pred_test_2[y_pred_test_2==-1]=1
        y_pred_val_2[y_pred_val_2==1]=0
        y_pred_val_2[y_pred_val_2==-1]=1
        average_precision_2_test= average_precision_score(y_test, y_pred_test_2)
        average_precision_2_val= average_precision_score(y_val, y_pred_val_2)
        print('Average precision-recall score test: {0:0.5f}'.format(average_precision_2_test))
        print('Average precision-recall score validation: {0:0.5f}'.format(average_precision_2_val))

[0.01]
[500, 1000, 2000, 3000, 4000, 5000, 10000]
contamination=0.01,n_estimators=500
Time for Isolation Forest fitting: 106.241
Average precision-recall score test: 0.15984
Average precision-recall score validation: 0.16506
contamination=0.01,n_estimators=1000
Time for Isolation Forest fitting: 207.008
Average precision-recall score test: 0.15181
Average precision-recall score validation: 0.16702
contamination=0.01,n_estimators=2000
Time for Isolation Forest fitting: 436.494
Average precision-recall score test: 0.14443
Average precision-recall score validation: 0.16539
contamination=0.01,n_estimators=3000
Time for Isolation Forest fitting: 621.962
Average precision-recall score test: 0.14424
Average precision-recall score validation: 0.16214
contamination=0.01,n_estimators=4000
Time for Isolation Forest fitting: 824.027
Average precision-recall score test: 0.14776
Average precision-recall score validation: 0.16571
contamination=0.01,n_estimators=5000
Time for Isolation Forest fitting:

In [75]:
#print("error train: %3f ; errors test : %3f ; "
#    "errors val: %3f"
#   % (n_error_train_2/np.shape(X_train)[0], n_error_test_2/np.shape(X_test)[0], n_error_val_2/np.shape(X_val)[0]))

In [76]:
y_pred_train_2[y_pred_train_2==1]=0
y_pred_train_2[y_pred_train_2==-1]=1
y_pred_test_2[y_pred_test_2==1]=0
y_pred_test_2[y_pred_test_2==-1]=1
y_pred_val_2[y_pred_val_2==1]=0
y_pred_val_2[y_pred_val_2==-1]=1

In [77]:
from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_val, y_pred_val_2))
print(roc_auc_score(y_test, y_pred_test_2))

0.633313946040417
0.629201989255337


In [82]:
average_precision_2_test= average_precision_score(y_test, y_pred_test_2)
average_precision_2_val= average_precision_score(y_val, y_pred_val_2)
print('Average precision-recall score test: {0:0.5f}'.format(
      average_precision_2_test))
print('Average precision-recall score validation: {0:0.5f}'.format(
      average_precision_2_val))

Average precision-recall score test: 0.12403
Average precision-recall score validation: 0.13345


In [79]:
from sklearn.metrics import f1_score
score_test_2= f1_score(y_test, y_pred_test_2)
score_val_2= f1_score(y_val, y_pred_val_2)
print('F1 Score test: %.5f' % score_test_2)
print('F1 Score validation test: %.5f' % score_val_2)

F1 Score test: 0.33247
F1 Score validation test: 0.34465


In [84]:
from sklearn.metrics import classification_report

print("Classification report for IF classifier %s:\n%s\n"
      % (clf2, classification_report(y_test, y_pred_test_2)))

Classification report for IF classifier IsolationForest(contamination=0.02, n_estimators=1000, random_state=42):
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.00      0.00      0.00     42647
         1.0       0.00      0.43      0.01       246

    accuracy                           0.00     42893
   macro avg       0.00      0.14      0.00     42893
weighted avg       0.00      0.00      0.00     42893




C:\Users\31639\anaconda3\envs\my environment 2021\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\31639\anaconda3\envs\my environment 2021\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\31639\anaconda3\envs\my environment 2021\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, l